# Modell 2 Alt: XGBoost-Modell mit Standardparametern
Eine veraltete Version von Modell 2 New ohne Hyperparametertuning oder Feature importance.

In [1]:
import os
import pandas as pd
import numpy as np
from xgboost import XGBRegressor
from tqdm import tqdm

# Funktion zur Datenbereinigung
def clean_data(df, is_train=True):
    """
    Bereinigt den DataFrame:
    - Ersetzt fehlende Werte (NaN) mit sinnvollen Standardwerten.
    - Entfernt problematische Spaltennamen.
    - Wandelt kategorische Spalten in numerische Werte um.
    """
    # Fehlende Werte auffüllen
    df.fillna(0, inplace=True)

    # Spaltennamen bereinigen
    df.columns = df.columns.str.replace(r"[^a-zA-Z0-9_]", "_", regex=True)

    # Sicherstellen, dass alle Spalten numerisch sind
    for col in df.columns:
        if df[col].dtype == 'object':
            df[col] = df[col].astype(str).astype('category').cat.codes

    return df


# Hauptskript
def main():
    # Aktuelles Arbeitsverzeichnis abrufen
    current_dir = os.getcwd().replace("\\", "/")

    # Trainings- und Testdaten einlesen
    train_path = os.path.join(current_dir, "data/train.csv")
    test_path = os.path.join(current_dir, "data/test.csv")

    df_train = pd.read_csv(train_path)
    df_test = pd.read_csv(test_path)

    # Daten bereinigen
    df_train = clean_data(df_train, is_train=True)
    df_test = clean_data(df_test, is_train=False)

    # Sicherstellen, dass die Spalten von df_train und df_test übereinstimmen
    missing_cols = set(df_train.columns) - set(df_test.columns)
    for col in missing_cols:
        df_test[col] = 0  # Fehlende Spalten in df_test hinzufügen und mit 0 füllen

    extra_cols = set(df_test.columns) - set(df_train.columns)
    df_test = df_test.drop(columns=extra_cols)  # Zusätzliche Spalten in df_test entfernen

    # Spaltenreihenfolge angleichen
    df_test = df_test[df_train.columns]

    # Features und Zielvariable definieren
    X_train = df_train.drop(columns=['efs', 'efs_time'], errors='ignore')
    y_train = df_train['efs']

    X_test = df_test.drop(columns=['efs', 'efs_time'], errors='ignore')

    # Sicherstellen, dass nur numerische Spalten verwendet werden
    X_train = X_train.select_dtypes(include=[np.number])
    X_test = X_test.select_dtypes(include=[np.number])

    # XGBRegressor initialisieren
    xgb_model = XGBRegressor(
        n_estimators=500,
        learning_rate=0.05,
        max_depth=5,
        random_state=42
    )

    # Training mit Fortschrittsbalken
    print("Training des Modells...")
    with tqdm(total=1, desc="Training") as pbar:
        xgb_model.fit(X_train, y_train)
        pbar.update(1)

    # Vorhersagen für Testdaten
    print("Vorhersagen für Testdaten...")
    risk_scores = xgb_model.predict(X_test)

    # Ergebnisse speichern
    results = pd.DataFrame({'ID': df_test['ID'], 'risk_score': risk_scores})
    results.to_csv(os.path.join(current_dir, "modell_2_old.csv"), index=False)
    print("✅ Ergebnisse wurden erfolgreich gespeichert: results.csv")


if __name__ == "__main__":
    main()


Training des Modells...


Training: 100%|███████████████████████████████████| 1/1 [01:02<00:00, 62.32s/it]

Vorhersagen für Testdaten...
✅ Ergebnisse wurden erfolgreich gespeichert: results.csv
